Group Details:

*   Janusz Feigel
*   Bhavana Malla
*   Brinda Rao

In [120]:
import os
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [121]:
print("TensorFlow version:", tf.__version__)
os.getcwd()

TensorFlow version: 2.6.0


'/content/drive/My Drive/Colab Notebooks'

In [122]:
os.chdir('/content/drive/My Drive/Colab Notebooks')
os.getcwd()

'/content/drive/My Drive/Colab Notebooks'

In [123]:
# Preprocessing the images and labels
def preprocess(images,labels):

  """Converting the images to float & Normailizing the images to range of [0,1]
   and reshaping to a single vector"""

  images = (images.astype(np.float32) / 255)
  # Make sure images have shape (28, 28, 1)
  images = np.expand_dims(images,-1)

  #Converting the labels to int 
  labels = labels.astype(np.int32)  
  # labels = tf.keras.utils.to_categorical(labels, 10) #num_classes = 10

  return images,labels

In [124]:
#Loading the MNIST Data
cifar = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()

In [125]:
#Preprocessing the labels and images
train_images,train_labels = preprocess(train_images,train_labels)
test_images,test_labels = preprocess(test_images,test_labels)
print("train_images shape: {} , test_images shape: {} " .format(train_images.shape,test_images.shape))

train_images shape: (60000, 28, 28, 1) , test_images shape: (10000, 28, 28, 1) 


In [126]:
#model configuration
n_h = 256
num_classes = 10
layer_list =  [
               tf.keras.Input(shape=(28, 28, 1), name="digits"),
               tf.keras.layers.Conv2D(n_h, kernel_size=(27, 27), activation="relu",padding='same'),
               tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
               tf.keras.layers.Conv2D(n_h//2, kernel_size=(27, 27), activation="relu",padding='same'),
               tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
               tf.keras.layers.Flatten(),
               tf.keras.layers.Dropout(0.2),
               tf.keras.layers.Dense(num_classes, activation="softmax"),
               ]
model = tf.keras.Sequential(layer_list)
model.summary()
optimizer = tf.optimizers.Adam(0.001)  # tune this
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 28, 28, 256)       186880    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 14, 128)       23888000  
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 6272)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 6272)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)              

In [127]:
#Creating the training and testing dataset using tensor slices
train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels
                                                 )).shuffle(60000).repeat(
                                                     ).batch(128)
# test_data = tf.data.Dataset.from_tensor_slices(
#     (test_images.reshape([-1, 784]).astype(np.float32) / 255, test_labels.astype(np.int32))).batch(128)

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images, test_labels)).batch(128)
                                                     

In [128]:
#Setting up log dir and file writer(s) for tensor board
from datetime import datetime
logdir = os.path.join("logs", "linear" + str(datetime.now()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

In [129]:
train_steps = 2500
learning_rate = 0.1

train_acc = []
test_acc = []
steps = []

for step, (img_batch, lbl_batch) in enumerate(train_data):
    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        logits = model(img_batch)
        # loss format is generally: first argument targets, second argument outputs
        xent = loss_fn(lbl_batch, logits)  
        
    #backpropagation
    variables = model.trainable_variables
    grads = tape.gradient(xent, variables)
    optimizer.apply_gradients(zip(grads, variables))
    
    train_acc_metric(lbl_batch, logits)

    # logging this stuff every time step
    with train_writer.as_default():
        tf.summary.scalar("loss", xent, step=step)
    
    if step % 100 == 0:
        # this is different from before. there, we only evaluated accuracy
        # for one batch. Now, we always average over 100 batches
        print("Step {} Loss: {} Train Accuracy: {}".format(
            step, xent, train_acc_metric.result()
            ))

        # log this only once every 100 steps
        with train_writer.as_default():
            tf.summary.scalar("Train Accuracy", train_acc_metric.result(), 
                              step=step)
            # tf.summary.image("Input", tf.reshape(img_batch, [-1, 28, 28, 1]),
                            #  step=step)


        

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


Step 0 Loss: 2.3034887313842773 Train Accuracy: 0.09375
Step 100 Loss: 0.16884155571460724 Train Accuracy: 0.7824876308441162
Step 200 Loss: 0.08025914430618286 Train Accuracy: 0.8721237778663635
Step 300 Loss: 0.2568831145763397 Train Accuracy: 0.9063019156455994
Step 400 Loss: 0.04230498522520065 Train Accuracy: 0.9246609807014465
Step 500 Loss: 0.0528297945857048 Train Accuracy: 0.9363460540771484
Step 600 Loss: 0.039492227137088776 Train Accuracy: 0.9444675445556641
Step 700 Loss: 0.06399330496788025 Train Accuracy: 0.9502830505371094
Step 800 Loss: 0.010141570121049881 Train Accuracy: 0.9549098610877991
Step 900 Loss: 0.1753142774105072 Train Accuracy: 0.9583709239959717
Step 1000 Loss: 0.023452676832675934 Train Accuracy: 0.9614760279655457
Step 1100 Loss: 0.14975175261497498 Train Accuracy: 0.9640951156616211
Step 1200 Loss: 0.026754537597298622 Train Accuracy: 0.9663626551628113
Step 1300 Loss: 0.04252266883850098 Train Accuracy: 0.9682576060295105
Step 1400 Loss: 0.01079269219

In [130]:
""" this is very convenient -- before, we usually had code that
         evaluates the whole test set at once -- this won't work for
         large datasets/models. With metrics, we can just iterate
         over the data and the metric takes care of averaging etc."""
        
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
    test_acc_metric(lbl_batch, model(img_batch))
    with test_writer.as_default():
      tf.summary.scalar("Test Accuracy", test_acc_metric.result(),
                              step=step)
print("Test accuracy: {}".format(test_acc_metric.result()))



Test accuracy: 0.9898999929428101


In [131]:
import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
# plt.xlabel("Steps")
# plt.ylabel("Accuracy")
# plt.plot(steps, train_acc_metric.result())
# plt.plot(steps, test_acc_metric.result())
# plt.legend(["Train", "Test"])


In [133]:
 %reload_ext tensorboard

In [134]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2040), started 1:23:26 ago. (Use '!kill 2040' to kill it.)

<IPython.core.display.Javascript object>